## Prefect

In [59]:
from prefect import task, Flow
from prefect.tasks.secrets import PrefectSecret
from prefect.tasks.postgres import PostgresExecute, PostgresFetch
import requests
import time
import yaml
import requests

In [60]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)
api_key = cfg['riotgamesapi']['key']
username = cfg['postgresql']['user']
password = cfg['postgresql']['password']
database = 'teamfighttactics'

/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_86021/2003926877.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [64]:
def get_users_from_league(league,password, riot_header, args):
    league_url = f"https://euw1.api.riotgames.com/tft/league/v1/{league}"
    league_response = requests.get(url=league_url, headers=riot_header).json()
    time.sleep(2)
    tier = league_response["tier"]
    for i in league_response["entries"]:
        summonerId = i["summonerId"]
        summonerName = i["summonerName"]
        leaguePoints = i["leaguePoints"]

        puuid_url = (
            f"https://euw1.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}"
        )
        puuid_response = requests.get(url=puuid_url, headers=riot_header).json()
        time.sleep(2)
        puuid = str(puuid_response["puuid"])

        user_query = """
            INSERT INTO public.users(
                id, summonername, summonerid, rankedleague, puuid, leaguepoints, region)
                VALUES (DEFAULT, %s, %s, %s, %s, %s , 'EUW1')
                """

        query_data = (
            summonerName,
            summonerId,
            tier,
            puuid,
            leaguePoints,
        )

        PostgresExecute(query=user_query, data=query_data, **args).run(
            password=password
        )


@task
def get_challenger_users(username, password, db_name, api_key):
    """A task that requires credentials to access something. Passing the
    credentials in as an argument allows you to change how/where the
    credentials are loaded (though we recommend using `PrefectSecret` tasks to
    load them."""

    riot_header = {"X-Riot-Token": api_key}

    args = {
        "db_name": db_name,
        "user": username,
        "host": "127.0.0.1",
        "port": 5432,
        "commit": True,
    }

    get_users_from_league(league="challenger", password=password, riot_header=riot_header, args=args)

@task
def get_grandmaster_users(username, password, db_name, api_key):

    riot_header = {"X-Riot-Token": api_key}

    args = {
        "db_name": db_name,
        "user": username,
        "host": "127.0.0.1",
        "port": 5432,
        "commit": True,
    }

    get_users_from_league(league="grandmaster", password=password, riot_header=riot_header, args=args)

In [65]:
with Flow("Get Grandmaster Summoners") as flow:
    username = PrefectSecret("USERNAME")
    password = PrefectSecret("PASSWORD")
    api_key = cfg['riotgamesapi']['key']
    dbname = "teamfighttactics"
    get_grandmaster_users(
        username=username, password=password, db_name=dbname, api_key=api_key
    )

flow.run()

[2021-10-30 11:31:31+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'Get Grandmaster Summoners'
[2021-10-30 11:31:31+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Starting task run...
[2021-10-30 11:31:31+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Finished task run for task with final state: 'Success'
[2021-10-30 11:31:31+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Starting task run...
[2021-10-30 11:31:31+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Finished task run for task with final state: 'Success'
[2021-10-30 11:31:31+0100] INFO - prefect.TaskRunner | Task 'get_grandmaster_users': Starting task run...
[2021-10-30 11:46:27+0100] INFO - prefect.TaskRunner | Task 'get_grandmaster_users': Finished task run for task with final state: 'Success'
[2021-10-30 11:46:27+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [80]:
def get__recent_match_history(puuid, password, riot_header, args):
    match_id_url = f"https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count=5"
    recent_match_ids = requests.get(url=match_id_url, headers=riot_header).json()
    time.sleep(2)

    match_id_query = "SELECT matchid FROM matchinfo WHERE matchid = ANY(%s);"
    match_id_data = (recent_match_ids,)
    existing_ids = PostgresFetch(
        query=match_id_query, data=match_id_data, fetch="all", **args
    ).run(password=password)

    existing_ids = [i[0] for i in existing_ids]
    new_matches = set(recent_match_ids).symmetric_difference(set(existing_ids))
    new_matches = list(new_matches)

    for match_id in new_matches:
        match_detail_url = (
            f"https://europe.api.riotgames.com/tft/match/v1/matches/{match_id}"
        )
        time.sleep(3)

        match_detail = requests.get(match_detail_url, headers=riot_header)
        match_detail = match_detail.json()

        for participant in match_detail["info"]["participants"]:

            user_id = participant['puuid']

            match_query = """
                INSERT INTO matchinfo VALUES (DEFAULT,%s,%s,%s)
                """

            match_data = (
                match_id,
                user_id,
                participant["placement"],
            )

            PostgresExecute(query=match_query, data=match_data, **args).run(
                password=password
            )

            participant_query = """
                SELECT id FROM matchinfo WHERE userid = %s and matchid = %s
                """

            participant_data = (user_id, match_id,)

            match_data_id = PostgresFetch(
                query=participant_query, data=participant_data, fetch="one", **args
            ).run(password=password)

            for units in participant['units']:
                character_id = units['character_id']
                tier = units['tier']
                items = units['items']
                item1 =  items[0] if len(items)>0 else None
                item2 = items[1] if len(items)>1 else None
                item3 = items[1] if len(items)>2 else None

            unit_query = """
                INSERT INTO matchunits VALUES (%s,%s,%s,%s,%s,%s)
                """
            unit_data = (match_data_id, character_id, tier, item1, item2, item3,)

            PostgresExecute(query=unit_query, data=unit_data, **args).run(
                password=password
            )

@task
def update_challenger_matches(username, password, db_name, api_key):
    """A task that requires credentials to access something. Passing the
    credentials in as an argument allows you to change how/where the
    credentials are loaded (though we recommend using `PrefectSecret` tasks to
    load them."""

    riot_header = {"X-Riot-Token": api_key}

    args = {
        "db_name": db_name,
        "user": username,
        "host": "127.0.0.1",
        "port": 5432,
        "commit": True,
    }

    user_query = '''SELECT puuid FROM users WHERE rankedleague = 'CHALLENGER' LIMIT 5 '''

    all_puuids = PostgresFetch(query=user_query, data=None, fetch="all", **args).run(
        password=password
    )

    for puuid in all_puuids:
        get__recent_match_history(puuid=puuid[0],password = password, riot_header=riot_header, args=args)
        
@task
def update_grandmaster_matches(username, password, db_name, api_key):
    """A task that requires credentials to access something. Passing the
    credentials in as an argument allows you to change how/where the
    credentials are loaded (though we recommend using `PrefectSecret` tasks to
    load them."""

    riot_header = {"X-Riot-Token": api_key}

    args = {
        "db_name": db_name,
        "user": username,
        "host": "127.0.0.1",
        "port": 5432,
        "commit": True,
    }

    user_query = '''SELECT puuid FROM users WHERE rankedleague = 'GRANDMASTER' LIMIT 5'''

    all_puuids = PostgresFetch(query=user_query, data=None, fetch="all", **args).run(
        password=password
    )

    for puuid in all_puuids:
        get__recent_match_history(puuid=puuid[0],password = password, riot_header=riot_header, args=args)


In [81]:
with Flow("Get Latest Challenger Matches") as flow:
    username = PrefectSecret("USERNAME")
    password = PrefectSecret("PASSWORD")
    api_key = cfg['riotgamesapi']['key']
    dbname = "teamfighttactics"
    update_challenger_matches(
        username=username, password=password, db_name=dbname, api_key=api_key
    )

flow.run()

[2021-10-30 12:11:31+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'Get Latest Challenger Matches'
[2021-10-30 12:11:31+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Starting task run...
[2021-10-30 12:11:31+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Finished task run for task with final state: 'Success'
[2021-10-30 12:11:31+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Starting task run...
[2021-10-30 12:11:31+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Finished task run for task with final state: 'Success'
[2021-10-30 12:11:31+0100] INFO - prefect.TaskRunner | Task 'update_challenger_matches': Starting task run...
[2021-10-30 12:11:31+0100] INFO - prefect.TaskRunner | Task 'update_challenger_matches': Finished task run for task with final state: 'Success'
[2021-10-30 12:11:31+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [82]:
with Flow("Get Latest Grandmaster Matches") as flow:
    username = PrefectSecret("USERNAME")
    password = PrefectSecret("PASSWORD")
    api_key = cfg['riotgamesapi']['key']
    dbname = "teamfighttactics"
    update_grandmaster_matches(
        username=username, password=password, db_name=dbname, api_key=api_key
    )

flow.run()

[2021-10-30 12:11:33+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'Get Latest Grandmaster Matches'
[2021-10-30 12:11:33+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Starting task run...
[2021-10-30 12:11:33+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Finished task run for task with final state: 'Success'
[2021-10-30 12:11:33+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Starting task run...
[2021-10-30 12:11:33+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Finished task run for task with final state: 'Success'
[2021-10-30 12:11:33+0100] INFO - prefect.TaskRunner | Task 'update_grandmaster_matches': Starting task run...
[2021-10-30 12:12:39+0100] INFO - prefect.TaskRunner | Task 'update_grandmaster_matches': Finished task run for task with final state: 'Success'
[2021-10-30 12:12:39+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">